In [ ]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 9.2 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# #Unzip test images folder
# !unzip /content/drive/MyDrive/MeeshoAIChallenge/test_images.zip -d /content/drive/MyDrive/MeeshoAIChallenge/

In [ ]:
import pandas as pd
import os
from PIL import Image
import csv

df = pd.read_parquet('/content/drive/MyDrive/MeeshoAIChallenge/category_attributes.parquet')
df

,Category,No_of_attribute,Attribute_list
0,Men Tshirts,5,"[color, neck, pattern, print_or_pattern_type, ..."
1,Sarees,10,"[blouse_pattern, border, border_width, color, ..."
2,Kurtis,9,"[color, fit_shape, length, occasion, ornamenta..."
3,Women Tshirts,8,"[color, fit_shape, length, pattern, print_or_p..."
4,Women Tops & Tunics,10,"[color, fit_shape, length, neck_collar, ocassi..."


In [ ]:
attr_mens_tshirt = list(df["Attribute_list"][0])
attr_sarees = list(df["Attribute_list"][1])
attr_kurtis = list(df["Attribute_list"][2])
attr_womens_tshirts = list(df["Attribute_list"][3])
attr_womens_tops_tunics = list(df["Attribute_list"][4])
attr_womens_tops_tunics[4] = "occasion"

In [ ]:
attr_sarees

['blouse_pattern',
 'border',
 'border_width',
 'color',
 'occasion',
 'ornamentation',
 'pallu_details',
 'pattern',
 'print_or_pattern_type',
 'transparency']

In [ ]:
men_tshirts_df = pd.read_csv("/content/drive/MyDrive/MeeshoAIChallenge/archive/Men_Tshirts.csv")
sarees_df = pd.read_csv("/content/drive/MyDrive/MeeshoAIChallenge/archive/Sarees.csv")
kurtis_df = pd.read_csv("/content/drive/MyDrive/MeeshoAIChallenge/archive/Kurtis.csv")
womens_tshirts_df = pd.read_csv("/content/drive/MyDrive/MeeshoAIChallenge/archive/Women_Tshirts.csv")
womens_tops_tunics_df = pd.read_csv("/content/drive/MyDrive/MeeshoAIChallenge/archive/Women_Tops_Tunics.csv")

sub_attr_men_tshirts = {attr: men_tshirts_df[attr].dropna().unique().tolist() for attr in attr_mens_tshirt}
sub_attr_sarees = {attr: sarees_df[attr].dropna().unique().tolist() for attr in attr_sarees}
sub_attr_kurtis = {attr: kurtis_df[attr].dropna().unique().tolist() for attr in attr_kurtis}
sub_attr_womens_tshirts = {attr: womens_tshirts_df[attr].dropna().unique().tolist() for attr in attr_womens_tshirts}
sub_attr_womens_tops_tunics = {attr: womens_tops_tunics_df[attr].dropna().unique().tolist() for attr in attr_womens_tops_tunics}


In [ ]:
sub_attr_sarees

{'blouse_pattern': ['same as saree', 'solid', 'same as border', 'default'],
 'border': ['woven design',
  'zari',
  'no border',
  'solid',
  'default',
  'temple border'],
 'border_width': ['small border', 'big border', 'no border'],
 'color': ['multicolor',
  'cream',
  'white',
  'default',
  'navy blue',
  'yellow',
  'green',
  'pink'],
 'occasion': ['party', 'traditional', 'daily', 'wedding'],
 'ornamentation': ['jacquard', 'default', 'tassels and latkans'],
 'pallu_details': ['woven design', 'same as saree', 'default', 'zari woven'],
 'pattern': ['zari woven', 'woven design', 'default', 'solid', 'printed'],
 'print_or_pattern_type': ['applique',
  'elephant',
  'floral',
  'ethnic motif',
  'peacock',
  'default',
  'solid',
  'checked',
  'botanical'],
 'transparency': ['no', 'yes']}

In [ ]:
df_test_mens_tshirts = pd.read_csv("/content/drive/MyDrive/MeeshoAIChallenge/archive2/test_Men_Tshirts.csv")
df_test_sarees = pd.read_csv("/content/drive/MyDrive/MeeshoAIChallenge/archive2/test_Sarees.csv")
df_test_kurtis = pd.read_csv("/content/drive/MyDrive/MeeshoAIChallenge/archive2/test_Kurtis.csv")
df_test_womens_tshirts = pd.read_csv("/content/drive/MyDrive/MeeshoAIChallenge/archive2/test_Women_Tshirts.csv")
df_test_womens_tops_tunics = pd.read_csv("/content/drive/MyDrive/MeeshoAIChallenge/archive2/test_Women_Tops_Tunics.csv")

In [ ]:
men_tshirts_df.head()

,id,Category,len,color,neck,pattern,print_or_pattern_type,sleeve_length,attr_6,attr_7,attr_8,attr_9,attr_10
0,0,Men Tshirts,5,default,round,printed,default,short sleeves,NaN,NaN,NaN,NaN,NaN
1,1,Men Tshirts,5,multicolor,polo,solid,solid,short sleeves,NaN,NaN,NaN,NaN,NaN
2,2,Men Tshirts,5,default,polo,solid,solid,short sleeves,NaN,NaN,NaN,NaN,NaN
3,3,Men Tshirts,5,multicolor,polo,solid,solid,short sleeves,NaN,NaN,NaN,NaN,NaN
4,4,Men Tshirts,5,multicolor,polo,solid,solid,short sleeves,NaN,NaN,NaN,NaN,NaN


In [ ]:
import torch
from PIL import Image
from transformers import AutoModel, AutoTokenizer

model = AutoModel.from_pretrained('openbmb/MiniCPM-Llama3-V-2_5', trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained('openbmb/MiniCPM-Llama3-V-2_5', trust_remote_code=True)
model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.91k [00:00<?, ?B/s]

configuration_minicpm.py:   0%|          | 0.00/4.06k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/openbmb/MiniCPM-Llama3-V-2_5-int4:
- configuration_minicpm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_minicpmv.py:   0%|          | 0.00/25.0k [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.9k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

MiniCPMV(
  (llm): LlamaForCausalLM(
    (model): LlamaModel(
      (embed_tokens): Embedding(128256, 4096)
      (layers): ModuleList(
        (0-31): 32 x LlamaDecoderLayer(
          (self_attn): LlamaSdpaAttention(
            (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
            (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
            (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
            (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
            (rotary_emb): LlamaRotaryEmbedding()
          )
          (mlp): LlamaMLP(
            (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
            (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
            (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
            (act_fn): SiLU()
          )
          (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      

In [ ]:
def generate_combined_question(image, attributes):
    question_parts = []
    for attr, options in attributes.items():
        options_text = ", ".join(options)
        question_parts.append(f"{attr} ({options_text})")

    question = (
        "Please describe the following attributes of this clothing item using exactly this format for each attribute -\n"
        "'Attribute: Attribute_Response'\n\n"
        "Attributes to describe: " + "; ".join(question_parts) + "."
    )
    question += "\nImportant: You must provide an answer for ALL attributes listed above, without missing any."

    system_prompt = (
        "You are an assistant helping to identify attributes of clothing items. "
        "You must follow these strict rules:\n"
        "1. Respond using ONLY this format for each attribute: 'Attribute: Attribute_Response'\n"
        "2. Provide answers for ALL attributes mentioned in the question\n"
        "3. Use only single-word responses from the provided options for each attribute\n"
        "4. Do not add any explanations, greetings, or additional text\n"

        "Example format:\n"
        "Color: Blue\n"
        "Pattern: Solid\n"
        "Style: Casual"
    )

    msgs = [{'role': 'user', 'content': question}]
    res = model.chat(
        image=image,
        msgs=msgs,
        tokenizer=tokenizer,
        sampling=True,
        temperature=0.5,
        system_prompt=system_prompt
    )
    return res

In [ ]:
csv_file_path = "/content/drive/MyDrive/MeeshoAIChallenge/output_mens_tshirts.csv"
category = "Men Tshirts"
length = 5

if not os.path.exists(csv_file_path):
    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file, quoting=csv.QUOTE_MINIMAL)
        writer.writerow(["id", "Category", "len", "response"])

for row in df_test_mens_tshirts[0:10].itertuples(index=False):
    image_id = str(row.id).zfill(6)
    image_path = f"/content/drive/MyDrive/MeeshoAIChallenge/test_images/{image_id}.jpg"
    print(image_path)

    if os.path.exists(image_path):
        raw_image = Image.open(image_path).convert('RGB')
        predicted_attributes = generate_combined_question(raw_image, sub_attr_men_tshirts)

        with open(csv_file_path, mode='a', newline='') as file:
            writer = csv.writer(file, quoting=csv.QUOTE_MINIMAL)
            writer.writerow([image_id, category, length, predicted_attributes])
    else:
        print(f"Image {image_id}.jpg not found.")

/content/drive/MyDrive/MeeshoAIChallenge/test_images/000000.jpg
Image 000000.jpg not found.
/content/drive/MyDrive/MeeshoAIChallenge/test_images/000001.jpg
Image 000001.jpg not found.
/content/drive/MyDrive/MeeshoAIChallenge/test_images/000002.jpg
Image 000002.jpg not found.
/content/drive/MyDrive/MeeshoAIChallenge/test_images/000003.jpg
Image 000003.jpg not found.
/content/drive/MyDrive/MeeshoAIChallenge/test_images/000004.jpg
Image 000004.jpg not found.
/content/drive/MyDrive/MeeshoAIChallenge/test_images/000005.jpg
Image 000005.jpg not found.
/content/drive/MyDrive/MeeshoAIChallenge/test_images/000006.jpg
Image 000006.jpg not found.
/content/drive/MyDrive/MeeshoAIChallenge/test_images/000007.jpg
Image 000007.jpg not found.
/content/drive/MyDrive/MeeshoAIChallenge/test_images/000008.jpg


RuntimeError: shape mismatch: value tensor of shape [1037] cannot be broadcast to indexing result of shape [1036]

In [ ]:
from transformers import AutoModel, AutoTokenizer
from PIL import Image
import torch
import csv
import os
from torchvision import transforms

# # Model and tokenizer setup
# model_path = "/content/fine-tuned-model-mens-tshirt"
# model = AutoModel.from_pretrained(model_path, trust_remote_code=True)
# tokenizer = AutoTokenizer.from_pretrained(model_path)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Preprocessing
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

csv_file_path = "/content/drive/MyDrive/MeeshoAIChallenge/output_mens_tshirts.csv"
category = "Men Tshirts"
length = 5

if not os.path.exists(csv_file_path):
    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file, quoting=csv.QUOTE_MINIMAL)
        writer.writerow(["id", "Category", "len", "response"])

for row in df_test_mens_tshirts[0:10].itertuples(index=False):
    image_id = str(row.id).zfill(6)
    image_path = f"/content/drive/MyDrive/MeeshoAIChallenge/test_images/{image_id}.jpg"
    print(image_path)

    if os.path.exists(image_path):
        raw_image = Image.open(image_path).convert('RGB')
        raw_image_tensor = preprocess(raw_image).unsqueeze(0).to(device)  # Preprocess and add batch dimension

        # Assuming generate_combined_question takes a tensor
        predicted_attributes = generate_combined_question(raw_image_tensor, sub_attr_men_tshirts)

        with open(csv_file_path, mode='a', newline='') as file:
            writer = csv.writer(file, quoting=csv.QUOTE_MINIMAL)
            writer.writerow([image_id, category, length, predicted_attributes])
    else:
        print(f"Image {image_id}.jpg not found.")


/content/drive/MyDrive/MeeshoAIChallenge/test_images/000000.jpg
Image 000000.jpg not found.
/content/drive/MyDrive/MeeshoAIChallenge/test_images/000001.jpg
Image 000001.jpg not found.
/content/drive/MyDrive/MeeshoAIChallenge/test_images/000002.jpg
Image 000002.jpg not found.
/content/drive/MyDrive/MeeshoAIChallenge/test_images/000003.jpg
Image 000003.jpg not found.
/content/drive/MyDrive/MeeshoAIChallenge/test_images/000004.jpg
Image 000004.jpg not found.
/content/drive/MyDrive/MeeshoAIChallenge/test_images/000005.jpg
Image 000005.jpg not found.
/content/drive/MyDrive/MeeshoAIChallenge/test_images/000006.jpg
Image 000006.jpg not found.
/content/drive/MyDrive/MeeshoAIChallenge/test_images/000007.jpg
Image 000007.jpg not found.
/content/drive/MyDrive/MeeshoAIChallenge/test_images/000008.jpg


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 13.06 MiB is free. Process 89621 has 14.73 GiB memory in use. Of the allocated memory 14.59 GiB is allocated by PyTorch, and 12.98 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
csv_file_path = "/content/drive/MyDrive/MeeshoAIChallenge/output_sarees.csv"
category = "Sarees"
length = 10

if not os.path.exists(csv_file_path):
    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file, quoting=csv.QUOTE_MINIMAL)
        writer.writerow(["id", "Category", "len", "response"])

for row in df_test_sarees.itertuples(index=False):
    image_id = str(row.id).zfill(6)
    image_path = f"/content/drive/MyDrive/MeeshoAIChallenge/test_images/{image_id}.jpg"
    print(image_path)

    if os.path.exists(image_path):
        raw_image = Image.open(image_path).convert('RGB')
        predicted_attributes = generate_combined_question(raw_image, sub_attr_sarees)

        with open(csv_file_path, mode='a', newline='') as file:
            writer = csv.writer(file, quoting=csv.QUOTE_MINIMAL)
            writer.writerow([image_id, category, length, predicted_attributes])
    else:
        print(f"Image {image_id}.jpg not found.")

In [ ]:
csv_file_path = "/content/drive/MyDrive/MeeshoAIChallenge/output_kurtis.csv"
category = "Kurtis"
length = 9

if not os.path.exists(csv_file_path):
    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file, quoting=csv.QUOTE_MINIMAL)
        writer.writerow(["id", "Category", "len", "response"])

for row in df_test_kurtis.itertuples(index=False):
    image_id = str(row.id).zfill(6)
    image_path = f"/content/drive/MyDrive/MeeshoAIChallenge/test_images/{image_id}.jpg"
    print(image_path)

    if os.path.exists(image_path):
        raw_image = Image.open(image_path).convert('RGB')
        predicted_attributes = generate_combined_question(raw_image, sub_attr_kurtis)

        with open(csv_file_path, mode='a', newline='') as file:
            writer = csv.writer(file, quoting=csv.QUOTE_MINIMAL)
            writer.writerow([image_id, category, length, predicted_attributes])
    else:
        print(f"Image {image_id}.jpg not found.")

In [ ]:
csv_file_path = "/content/drive/MyDrive/MeeshoAIChallenge/output_womens_tshirts.csv"
category = "Women Tshirts"
length = 8

if not os.path.exists(csv_file_path):
    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file, quoting=csv.QUOTE_MINIMAL)
        writer.writerow(["id", "Category", "len", "response"])

for row in df_test_womens_tshirts.itertuples(index=False):
    image_id = str(row.id).zfill(6)
    image_path = f"/content/drive/MyDrive/MeeshoAIChallenge/test_images/{image_id}.jpg"
    print(image_path)

    if os.path.exists(image_path):
        raw_image = Image.open(image_path).convert('RGB')
        predicted_attributes = generate_combined_question(raw_image, sub_attr_womens_tshirts)

        with open(csv_file_path, mode='a', newline='') as file:
            writer = csv.writer(file, quoting=csv.QUOTE_MINIMAL)
            writer.writerow([image_id, category, length, predicted_attributes])
    else:
        print(f"Image {image_id}.jpg not found.")

In [ ]:
csv_file_path = "/content/drive/MyDrive/MeeshoAIChallenge/output_womens_tops_tunics.csv"
category = "Women Tops & Tunics"
length = 10

if not os.path.exists(csv_file_path):
    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file, quoting=csv.QUOTE_MINIMAL)
        writer.writerow(["id", "Category", "len", "response"])

for row in df_test_womens_tops_tunics.itertuples(index=False):
    image_id = str(row.id).zfill(6)
    image_path = f"/content/drive/MyDrive/MeeshoAIChallenge/test_images/{image_id}.jpg"
    print(image_path)

    if os.path.exists(image_path):
        raw_image = Image.open(image_path).convert('RGB')
        predicted_attributes = generate_combined_question(raw_image, sub_attr_womens_tops_tunics)

        with open(csv_file_path, mode='a', newline='') as file:
            writer = csv.writer(file, quoting=csv.QUOTE_MINIMAL)
            writer.writerow([image_id, category, length, predicted_attributes])
    else:
        print(f"Image {image_id}.jpg not found.")

/content/drive/MyDrive/MeeshoAIChallenge/test_images/023564.jpg
/content/drive/MyDrive/MeeshoAIChallenge/test_images/023565.jpg
/content/drive/MyDrive/MeeshoAIChallenge/test_images/023566.jpg
/content/drive/MyDrive/MeeshoAIChallenge/test_images/023567.jpg


KeyboardInterrupt: 

In [ ]:
!pip install transformers datasets accelerate peft --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 333.2/333.2 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from datasets import Dataset
from peft import LoraConfig, get_peft_model, TaskType

In [ ]:
model_name = 'openbmb/MiniCPM-Llama3-V-2_5-int4'

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)
model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

MiniCPMV(
  (llm): LlamaForCausalLM(
    (model): LlamaModel(
      (embed_tokens): Embedding(128256, 4096)
      (layers): ModuleList(
        (0-31): 32 x LlamaDecoderLayer(
          (self_attn): LlamaSdpaAttention(
            (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
            (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
            (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
            (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
            (rotary_emb): LlamaRotaryEmbedding()
          )
          (mlp): LlamaMLP(
            (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
            (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
            (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
            (act_fn): SiLU()
          )
          (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      

In [ ]:
import pandas as pd

# Load your DataFrame (replace this with your actual DataFrame)
men_tshirts_df = pd.read_csv("/content/drive/MyDrive/MeeshoAIChallenge/archive/Men_Tshirts.csv")

# Create input-output pairs
def create_prompt(row):
    attributes = ', '.join([
        str(row[col]) for col in ['color', 'neck', 'pattern', 'print_or_pattern_type', 'sleeve_length']
        if pd.notna(row[col])
    ])
    prompt = f"Describe a men's t-shirt with the following attributes: {attributes}."
    return prompt

def create_output(row):
    # If you have actual descriptions, use them. Otherwise, generate synthetic ones.
    return "This is a sample description for the t-shirt."

men_tshirts_df['input'] = men_tshirts_df.apply(create_prompt, axis=1)
men_tshirts_df['output'] = men_tshirts_df.apply(create_output, axis=1)


In [ ]:
def create_synthetic_description(row):
    # Extract attributes, handling missing values
    attributes = []
    if pd.notna(row['color']) and row['color'] != 'default':
        attributes.append(f"{row['color']}")
    if pd.notna(row['neck']) and row['neck'] != 'default':
        attributes.append(f"{row['neck']} neck")
    if pd.notna(row['pattern']) and row['pattern'] != 'default':
        attributes.append(f"{row['pattern']} pattern")
    if pd.notna(row['print_or_pattern_type']) and row['print_or_pattern_type'] != 'default':
        attributes.append(f"{row['print_or_pattern_type']} type")
    if pd.notna(row['sleeve_length']) and row['sleeve_length'] != 'default':
        attributes.append(f"{row['sleeve_length']}")

    # Construct the description
    description = "A men's t-shirt featuring " + ', '.join(attributes) + '.'
    return description


In [ ]:
men_tshirts_df['output'] = men_tshirts_df.apply(create_synthetic_description, axis=1)


In [ ]:
def create_input_prompt(row):
    attributes = ', '.join([
        f"{col}: {row[col]}" for col in ['color', 'neck', 'pattern', 'print_or_pattern_type', 'sleeve_length']
        if pd.notna(row[col]) and row[col] != 'default'
    ])
    prompt = f"Describe a men's t-shirt with the following attributes: {attributes}."
    return prompt

men_tshirts_df['input'] = men_tshirts_df.apply(create_input_prompt, axis=1)


In [ ]:
# Display the DataFrame
print(men_tshirts_df[['input', 'output']])

                                                  input  \
0     Describe a men's t-shirt with the following at...   
1     Describe a men's t-shirt with the following at...   
2     Describe a men's t-shirt with the following at...   
3     Describe a men's t-shirt with the following at...   
4     Describe a men's t-shirt with the following at...   
...                                                 ...   
7262  Describe a men's t-shirt with the following at...   
7263  Describe a men's t-shirt with the following at...   
7264  Describe a men's t-shirt with the following at...   
7265  Describe a men's t-shirt with the following at...   
7266  Describe a men's t-shirt with the following at...   

                                                 output  
0     A men's t-shirt featuring round neck, printed ...  
1     A men's t-shirt featuring multicolor, polo nec...  
2     A men's t-shirt featuring polo neck, solid pat...  
3     A men's t-shirt featuring multicolor, polo nec...  
4

In [ ]:
from datasets import Dataset

dataset = Dataset.from_pandas(men_tshirts_df[['input', 'output']])


In [ ]:
def tokenize_function(examples):
    model_inputs = tokenizer(
        examples['input'],
        truncation=True,
        max_length=512,
        padding='max_length'
    )
    labels = tokenizer(
        examples['output'],
        truncation=True,
        max_length=512,
        padding='max_length'
    )['input_ids']
    model_inputs['labels'] = labels
    return model_inputs

tokenized_datasets = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/7267 [00:00<?, ? examples/s]

In [ ]:
# from peft import LoraConfig, get_peft_model, TaskType

# peft_config = LoraConfig(
#     task_type=TaskType.CAUSAL_LM,
#     inference_mode=False,
#     r=8,
#     lora_alpha=32,
#     lora_dropout=0.1,
# )

from peft import LoraConfig, get_peft_model, TaskType

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=['attention.query', 'attention.key', 'attention.value', 'mlp.fc_in', 'mlp.fc_out']
)


model = get_peft_model(model, peft_config)


ValueError: Target modules {'attention.query', 'attention.value', 'attention.key', 'mlp.fc_in', 'mlp.fc_out'} not found in the base model. Please check the target modules and try again.

In [ ]:
for name, module in model.named_modules():
    print(name)


llm
llm.model
llm.model.embed_tokens
llm.model.layers
llm.model.layers.0
llm.model.layers.0.self_attn
llm.model.layers.0.self_attn.q_proj
llm.model.layers.0.self_attn.k_proj
llm.model.layers.0.self_attn.v_proj
llm.model.layers.0.self_attn.o_proj
llm.model.layers.0.self_attn.rotary_emb
llm.model.layers.0.mlp
llm.model.layers.0.mlp.gate_proj
llm.model.layers.0.mlp.up_proj
llm.model.layers.0.mlp.down_proj
llm.model.layers.0.mlp.act_fn
llm.model.layers.0.input_layernorm
llm.model.layers.0.post_attention_layernorm
llm.model.layers.1
llm.model.layers.1.self_attn
llm.model.layers.1.self_attn.q_proj
llm.model.layers.1.self_attn.k_proj
llm.model.layers.1.self_attn.v_proj
llm.model.layers.1.self_attn.o_proj
llm.model.layers.1.self_attn.rotary_emb
llm.model.layers.1.mlp
llm.model.layers.1.mlp.gate_proj
llm.model.layers.1.mlp.up_proj
llm.model.layers.1.mlp.down_proj
llm.model.layers.1.mlp.act_fn
llm.model.layers.1.input_layernorm
llm.model.layers.1.post_attention_layernorm
llm.model.layers.2
llm.

In [ ]:
# Define the target modules
target_modules = ['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj']

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=target_modules
)


In [ ]:
model = get_peft_model(model, peft_config)


In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=2,
    num_train_epochs=10,
    learning_rate=1e-4,
    fp16=True,
    logging_steps=10,
    save_steps=500,
    evaluation_strategy='no',
    report_to='none',
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
)


In [ ]:
def data_collator(features):
    # Convert list of features to dict of lists
    batch = {}
    for key in features[0]:
        batch[key] = torch.stack([f[key] for f in features])

    # Package inputs into 'data' as expected by MiniCPMV
    return {'data': batch}


In [ ]:
# trainer.train()

# # Save the Fine-Tuned Model


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    data_collator=data_collator,
)

trainer.save_model('./fine-tuned-model-mens-tshirt')
tokenizer.save_pretrained('./fine-tuned-model-mens-tshirt')

# Generate Descriptions with the Fine-Tuned Model
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MiniCPMV(
      (llm): LlamaForCausalLM(
        (model): LlamaModel(
          (embed_tokens): Embedding(128256, 4096)
          (layers): ModuleList(
            (0-31): 32 x LlamaDecoderLayer(
              (self_attn): LlamaSdpaAttention(
                (q_proj): lora.Linear4bit(
                  (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.1, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=4096, out_features=8, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default): Linear(in_features=8, out_features=4096, bias=False)
                  )
                  (lora_embedding_A): ParameterDict()
                  (lora_embedding_B): ParameterDict()
                  (lora_magnitude_vec

In [ ]:
# Tokenize the input prompt
test_prompt = "Describe a men's t-shirt with the following attributes: color: blue, neck: V-neck, pattern: striped, sleeve_length: long sleeves."
inputs = tokenizer(test_prompt)

# Prepare input IDs as expected by the model
input_id_list = [inputs['input_ids']]  # Model may expect a list

# Move model to device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Generate text
with torch.no_grad():
    outputs = model.generate(
        input_id_list=input_id_list,
        img_list=None,                    # Assuming no images
        tokenizer=tokenizer,
        max_new_tokens=50,                # Use max_new_tokens
        stream=False
    )

# Inspect outputs
print("outputs:", outputs)
print("Type of outputs:", type(outputs))
print("Type of outputs[0]:", type(outputs[0]))

# Decode or use the output based on its type
if isinstance(outputs[0], str):
    generated_text = outputs[0]
elif isinstance(outputs[0], (list, torch.Tensor)):
    if isinstance(outputs[0], torch.Tensor):
        token_ids = outputs[0].tolist()
    else:
        token_ids = outputs[0]
    generated_text = tokenizer.decode(token_ids, skip_special_tokens=True)
else:
    raise TypeError(f"Unexpected output type: {type(outputs[0])}")

print("Generated Text:", generated_text)


outputs: ["The men's t-shirt is blue in color, has a V-neck, is striped, and has long sleeves."]
Type of outputs: <class 'list'>
Type of outputs[0]: <class 'str'>
Generated Text: The men's t-shirt is blue in color, has a V-neck, is striped, and has long sleeves.


In [ ]:
!zip -r /content/finetuned.zip /content/fine-tuned-model-mens-tshirt

  adding: content/fine-tuned-model-mens-tshirt/ (stored 0%)
  adding: content/fine-tuned-model-mens-tshirt/README.md (deflated 66%)
  adding: content/fine-tuned-model-mens-tshirt/training_args.bin (deflated 51%)
  adding: content/fine-tuned-model-mens-tshirt/adapter_model.safetensors (deflated 56%)
  adding: content/fine-tuned-model-mens-tshirt/tokenizer.json (deflated 85%)
  adding: content/fine-tuned-model-mens-tshirt/adapter_config.json (deflated 53%)
  adding: content/fine-tuned-model-mens-tshirt/special_tokens_map.json (deflated 72%)
  adding: content/fine-tuned-model-mens-tshirt/tokenizer_config.json (deflated 96%)


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# Load the model and tokenizer
model_path = "/path/to/your/fine-tuned-model-mens-ts"  # Adjust to the actual path of your folder
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(
    model_path,
    from_safetensors=True  # Indicating that safetensors format is being used
)

# Function to run inference
def predict(text):
    inputs = tokenizer(text, return_tensors="pt")  # Encode text to tensors
    with torch.no_grad():  # Disable gradient calculations for inference
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    return predicted_class, logits

# Example usage
text = "Sample input text for model inference"
predicted_class, logits = predict(text)
print(f"Predicted class: {predicted_class}")
print(f"Logits: {logits}")


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from PIL import Image
import torch
from torchvision import transforms

# Load the model and tokenizer
model_path = "/content/fine-tuned-model-mens-tshirt"  # Adjust this path
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_path,
    from_safetensors=True,  # Indicate safetensors format
    trust_remote_code=True
)

# Move model to device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

The repository for /content/fine-tuned-model-mens-tshirt contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co//content/fine-tuned-model-mens-tshirt.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


ValueError: Unrecognized configuration class <class 'transformers_modules.openbmb.MiniCPM-Llama3-V-2_5-int4.028d758be0c431f9cf79c859b7124e12aaed5129.configuration_minicpm.MiniCPMVConfig'> for this kind of AutoModel: AutoModelForSeq2SeqLM.
Model type should be one of BartConfig, BigBirdPegasusConfig, BlenderbotConfig, BlenderbotSmallConfig, EncoderDecoderConfig, FSMTConfig, GPTSanJapaneseConfig, LEDConfig, LongT5Config, M2M100Config, MarianConfig, MBartConfig, MT5Config, MvpConfig, NllbMoeConfig, PegasusConfig, PegasusXConfig, PLBartConfig, ProphetNetConfig, Qwen2AudioConfig, SeamlessM4TConfig, SeamlessM4Tv2Config, SwitchTransformersConfig, T5Config, UMT5Config, XLMProphetNetConfig.

In [ ]:
from transformers import AutoModel
model_path = "/content/fine-tuned-model-mens-tshirt"
tokenizer = AutoTokenizer.from_pretrained(model_path)

model = AutoModel.from_pretrained(
    model_path,
    trust_remote_code=True
)

The repository for /content/fine-tuned-model-mens-tshirt contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co//content/fine-tuned-model-mens-tshirt.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 1002.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 13.06 MiB is free. Process 89621 has 14.73 GiB memory in use. Of the allocated memory 14.59 GiB is allocated by PyTorch, and 13.03 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)